In [ ]:
# from dotenv import load_dotenv,find_dotenv
import os
import pprint
import pymongo
from pymongo import MongoClient



In [3]:

client = MongoClient("mongodb://localhost:27017/")

In [4]:
production=client.production

In [5]:
book_validator = {
    "$jsonSchema": {
        "bsonType": "object",
        "required": ["title", "authors", "publish_date", "type", "copies"],
        "properties": {
            "title": {
                "bsonType": "string",
                "description": "must be a string and is required"
            },
            "authors": {
                "bsonType": "array",
                "items": {
                    "bsonType": "objectId",
                    "description": "must be an objectId and is required"
                }
            },
            "publish_date": {
                "bsonType": "date",
                "description": "must be a date and is required"
            },
            "type": {
                "enum": ["Fiction", "Non-Fiction"],
                "description": "can only be one of the enum values and is required"
            },
            "copies": {
                "bsonType": "int",
                "minimum": 0,
                "description": "must be an integer greater than 0 and is required"
            }
        }
    }
}


In [6]:
try:
    production.create_collection("book")
except Exception as e:
    print(e)
    
production.command("collMod","book",validator=book_validator)


{'ok': 1.0}

In [8]:
def create_author_collection():
    author_validator = {
        "$jsonSchema":{
            "bsonType":"object",
            "required":["first_name","last_name","date_of_birth"],
            "properties":{
                "first_name":{
                    "bsonType":"string",
                    "description":"must be a string and is required"
                },
                "last_name":{
                    "bsonType":"string",
                    "description":"must be a string and is required"
                },
                "date_of_birth":{
                    "bsonType":"date",
                    "description":"must be a string and is required"
                }
            }
        }
    }
    
    try:
        production.create_collection("author")
    except Exception as e:
        print(e)
    
    
    production.command("collMod","author",validator=author_validator)


create_author_collection()

collection author already exists


In [11]:
from datetime import datetime as dt


In [18]:
authors=[
    {
        "first_name":"Tim",
        "last_name":"Holy",
        "date_of_birth":dt(1975,10,27)
    },
    {
        "first_name":"Stephen",
        "last_name":"Hawking",
        "date_of_birth":dt(1942,1,8)
    },
    {
        "first_name":"Carl",
        "last_name":"Sagan",
        "date_of_birth":dt(1934,11,9)
    },
    {
        "first_name":"Jane",
        "last_name":"Austen",
        "date_of_birth":dt(1775,12,16)
    },
    {
        "first_name":"Mark",
        "last_name":"Twain",
        "date_of_birth":dt(1835,11,30)
    },
    {
        "first_name":"Agatha",
        "last_name":"Christie", 
        "date_of_birth":dt(1890,9,15)
    }
]
# /author_collection=production.author
# author_collection.insert_many(authors)

authors=author_collection.insert_many(authors).inserted_ids

In [19]:
for author in author_collection.find():
    print(author)

{'_id': ObjectId('67bb3ea110d1e4fd855d7f2e'), 'first_name': 'Tim', 'last_name': 'Holy', 'date_of_birth': datetime.datetime(1975, 10, 27, 0, 0)}
{'_id': ObjectId('67bb3ea110d1e4fd855d7f2f'), 'first_name': 'Stephen', 'last_name': 'Hawking', 'date_of_birth': datetime.datetime(1942, 1, 8, 0, 0)}
{'_id': ObjectId('67bb3ea110d1e4fd855d7f30'), 'first_name': 'Carl', 'last_name': 'Sagan', 'date_of_birth': datetime.datetime(1934, 11, 9, 0, 0)}
{'_id': ObjectId('67bb3ea110d1e4fd855d7f31'), 'first_name': 'Jane', 'last_name': 'Austen', 'date_of_birth': datetime.datetime(1775, 12, 16, 0, 0)}
{'_id': ObjectId('67bb3ea110d1e4fd855d7f32'), 'first_name': 'Mark', 'last_name': 'Twain', 'date_of_birth': datetime.datetime(1835, 11, 30, 0, 0)}
{'_id': ObjectId('67bb3ea110d1e4fd855d7f33'), 'first_name': 'Agatha', 'last_name': 'Christie', 'date_of_birth': datetime.datetime(1890, 9, 15, 0, 0)}


In [16]:
author_collection.delete_many({})


DeleteResult({'n': 12, 'ok': 1.0}, acknowledged=True)

In [17]:
for author in author_collection.find():
    print(author)

In [23]:
authors_list = list(author_collection.find())

# Create books with proper references to authors
books = [
    {
        "title": "Image Processing in Julia",
        "authors": [authors_list[0]['_id']],  # Tim Holy
        "publish_date": dt(2020, 5, 15),
        "type": "Non-Fiction",
        "copies": 100
    },
    {
        "title": "A Brief History of Time",
        "authors": [authors_list[1]['_id']],  # Stephen Hawking
        "publish_date": dt(1988, 3, 1),
        "type": "Non-Fiction",
        "copies": 500
    },
    {
        "title": "Cosmos",
        "authors": [authors_list[2]['_id']],  # Carl Sagan
        "publish_date": dt(1980, 9, 28),
        "type": "Non-Fiction",
        "copies": 300
    },
    {
        "title": "Pride and Prejudice",
        "authors": [authors_list[3]['_id']],  # Jane Austen
        "publish_date": dt(1813, 1, 28),
        "type": "Fiction",
        "copies": 1000
    },
    {
        "title": "The Adventures of Tom Sawyer",
        "authors": [authors_list[4]['_id']],  # Mark Twain
        "publish_date": dt(1876, 6, 1),
        "type": "Fiction",
        "copies": 750
    },
    {
        "title": "Murder on the Orient Express",
        "authors": [authors_list[5]['_id']],  # Agatha Christie
        "publish_date": dt(1934, 1, 1),
        "type": "Fiction",
        "copies": 400
    }
]
try:
    result = production.book.insert_many(books)
    print("Books added successfully!")
    
    # Let's see our books with author names
    for book in production.book.find():
        author = author_collection.find_one({"_id": book['authors'][0]})
        print(f"\nTitle: {book['title']}")
        print(f"Author: {author['first_name']} {author['last_name']}")
        print(f"Type: {book['type']}")
        print(f"Copies: {book['copies']}")
except Exception as e:
    print(f"Error: {e}")

Books added successfully!

Title: Image Processing in Julia
Author: Tim Holy
Type: Non-Fiction
Copies: 100

Title: A Brief History of Time
Author: Stephen Hawking
Type: Non-Fiction
Copies: 500

Title: Cosmos
Author: Carl Sagan
Type: Non-Fiction
Copies: 300

Title: Pride and Prejudice
Author: Jane Austen
Type: Fiction
Copies: 1000

Title: The Adventures of Tom Sawyer
Author: Mark Twain
Type: Fiction
Copies: 750

Title: Murder on the Orient Express
Author: Agatha Christie
Type: Fiction
Copies: 400


In [25]:

print(f"Number of books inserted: {len(result.inserted_ids)}")

# Double check the count
count = production.book.count_documents({})
print(f"Total books in collection: {count}")

Number of books inserted: 6
Total books in collection: 6


In [22]:

production.book.delete_many({})


DeleteResult({'n': 12, 'ok': 1.0}, acknowledged=True)

In [31]:
authors_and_books = production.author.aggregate([{
    "$lookup":{
        "from":"book",
        "localField":"_id",
        "foreignField":"authors",
        "as":"books"
    }
}])


In [32]:
pprint.pprint(list(authors_and_books))

[{'_id': ObjectId('67bb3ea110d1e4fd855d7f2e'),
  'books': [{'_id': ObjectId('67bb419a10d1e4fd855d7f40'),
             'authors': [ObjectId('67bb3ea110d1e4fd855d7f2e')],
             'copies': 100,
             'publish_date': datetime.datetime(2020, 5, 15, 0, 0),
             'title': 'Image Processing in Julia',
             'type': 'Non-Fiction'}],
  'date_of_birth': datetime.datetime(1975, 10, 27, 0, 0),
  'first_name': 'Tim',
  'last_name': 'Holy'},
 {'_id': ObjectId('67bb3ea110d1e4fd855d7f2f'),
  'books': [{'_id': ObjectId('67bb419a10d1e4fd855d7f41'),
             'authors': [ObjectId('67bb3ea110d1e4fd855d7f2f')],
             'copies': 500,
             'publish_date': datetime.datetime(1988, 3, 1, 0, 0),
             'title': 'A Brief History of Time',
             'type': 'Non-Fiction'}],
  'date_of_birth': datetime.datetime(1942, 1, 8, 0, 0),
  'first_name': 'Stephen',
  'last_name': 'Hawking'},
 {'_id': ObjectId('67bb3ea110d1e4fd855d7f30'),
  'books': [{'_id': ObjectId('67bb4